# 1 - Simple Sentiment Analysis

Source: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 39.1MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
for i in range(5):
  print(vars(train_data.examples[i]))

{'text': ['More', 'than', 'twenty', 'years', 'before', 'Peter', 'Jackson', "'s", 'visionary', 'adaptation', 'of', 'The', 'Lord', 'Of', 'The', 'Rings', ',', 'there', 'was', 'this', '1978', 'animated', 'effort', 'from', 'director', 'Ralph', 'Bakshi', '.', 'An', 'ambitious', 'and', 'reasonably', 'faithful', 'version', 'of', 'the', 'story', ',', 'this', 'has', 'sadly', 'been', 'rather', 'over', '-', 'shadowed', 'by', 'the', 'Jackson', 'trilogy', '.', 'Indeed', ',', 'many', 'reviewers', 'here', 'on', 'the', 'IMDb', '(', 'mainly', 'those', 'who', 'saw', 'the', 'newer', 'version', 'first', ')', 'seem', 'to', 'be', 'fiercely', 'unkind', 'to', 'this', 'version', '....', 'but', 'if', 'one', 'applies', 'a', 'little', 'common', 'sense', ',', 'and', 'takes', 'into', 'consideration', 'the', 'time', 'when', 'it', 'was', 'made', 'and', 'the', 'technical', 'possibilities', 'that', 'existed', 'at', 'that', 'time', ',', 'then', 'they', 'will', 'realise', 'that', 'this', 'is', 'a', 'pretty', 'good', 'film

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 26_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 26002
Unique tokens in LABEL vocabulary: 2


In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203100), (',', 194023), ('.', 165910), ('and', 109996), ('a', 109807), ('of', 100728), ('to', 93613), ('is', 76386), ('in', 61383), ('I', 54592), ('it', 53478), ('that', 49099), ('"', 44398), ("'s", 43622), ('this', 42349), ('-', 37158), ('/><br', 35545), ('was', 35105), ('as', 30646), ('with', 30131)]


In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [13]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fad9c83cbf8>, {'neg': 0, 'pos': 1})


In [0]:

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,692,105 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
#The BCEWithLogitsLoss criterion carries out both the sigmoid and the binary cross entropy steps.
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #preds[batch_size,]
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        #prediction [batch_size,1]
        predictions = model(batch.text).squeeze(1)
        #prediction [batch_size,1]
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.695 | Train Acc: 50.32%
	 Val. Loss: 0.694 |  Val. Acc: 50.36%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.01%
	 Val. Loss: 0.694 |  Val. Acc: 51.41%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.13%
	 Val. Loss: 0.694 |  Val. Acc: 51.61%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.75%
	 Val. Loss: 0.694 |  Val. Acc: 50.12%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.01%
	 Val. Loss: 0.694 |  Val. Acc: 51.99%


In [28]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.691 | Test Acc: 53.18%
